This is the code with which the "NQ dev ds with answers" has been generated.

In [2]:
resetDb()

# Wikipedia Parser
Parse and structures the NQ-dev dataset Wikipedia pages to create the graph that will be used as the KB of the experiment.

In [ ]:
from utils.wikipediaParser import *
import io
from tqdm import tqdm
import pandas as pd
import config.EnvLoader as el

In [2]:
resetCsv()

In [3]:
parsed_urls = []
parsed_titles = []
no_answer = []


with open("v1.0-simplified_nq-dev-all.jsonl") as file:
    for line in tqdm(file):
        line = io.StringIO(line)
        jsonObj = pd.read_json(path_or_buf=line, lines=True)
        # If the document has already been parsed, skip
        if jsonObj['document_url'][0] in parsed_urls or jsonObj['document_title'][0] in parsed_titles:
            continue
        # If the document does not have an answer, skip
        ann_lst = [ann['long_answer']['start_byte'] for ann in jsonObj['annotations'][0]]
        if len(set(ann_lst)) == 1 and ann_lst[0] == -1:
            no_answer.append(jsonObj["question_text"][0])
            continue
        parse(jsonObj.iloc[0])
        parsed_urls.append(jsonObj["document_url"][0])
        parsed_titles.append(jsonObj["document_title"][0])

0it [00:00, ?it/s]

7830it [08:04, 16.17it/s]


In [4]:
len(no_answer)

2202

Move csvs into db's import folder

In [3]:
createConstrains()

Creating Constrain 1/4
Creating Constrain 2/4
Creating Contrain 3/4
Creating Contrain 4/4


In [4]:
query_lst = neo4jExecutor()

Creating :Page nodes
Creating :SubChapter nodes
Creating :Chunk nodes
Linking subchapters
Linking chunks
Linking main pages
Linking cited pages
Creating previous and next relationships
Linking Pages to all chunks
Linking SubChapters to all chunks
Creating OriginalPage label 1/2
Creating OriginalPage label 2/2


In [5]:
linkChunks()

100%|██████████| 4897/4897 [00:58<00:00, 83.57it/s] 


Move to DBpedia DMBS and run

In [11]:
extractRelations(query_lst)
extractLabels(query_lst)

found 349502 DBpedia labels


Copy DBpedia csvs and move back to the db

In [12]:
addDBpediaInfo()

Adding DBpedia relationships
Adding DBpedia labels


In [ ]:
# Extract answers
URI = "neo4j://localhost"
AUTH = ("neo4j", el.NEO4J_PWD)
result = []

with open("v1.0-simplified_nq-dev-all.jsonl") as file:
    for line in tqdm(file):
        line = io.StringIO(line)
        jsonObj = pd.read_json(path_or_buf=line, lines=True)
        result.append(extractAnswers(jsonObj, URI, AUTH))

0it [00:00, ?it/s]

4705it [01:32, 50.01it/s]

Couldn't find 0


7830it [02:36, 50.17it/s]


In [3]:
df_result = pd.DataFrame(result, columns = ['question', 'result']) 
print("Pages not found in Neo4j: " + str(len(df_result[df_result["result"] == "Page not found in Neo4j"])))
print("Questions with no answers: " + str(len(df_result[df_result["result"] == "No answer provided"])))
print("Non-found chunks: " + str(len(df_result[df_result["result"] == "not found"])))
print("Found answers: " + str(len(df_result[df_result["result"] == "found"])))


Pages not found in Neo4j: 1
Questions with no answers: 2331
Non-found chunks: 19
Found answers: 5479


In [ ]:
# Import answers
writeAnswers(URI, AUTH)

In [41]:
# Remove non-requested DBpedia relaitonships
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    pair_lst, summary, keys = driver.execute_query(
        """CALL apoc.periodic.iterate(
        "MATCH (a:Page)-[r:vector_rel]-(b:Page)
        WHERE NOT (a)-[:cited_in]->(:Chunk)-[:same_page]->(:Chunk)-[:cites]->(b)
        AND NOT (a)-[:has_chunk]->(:Chunk)-[:cites]->(b)
        AND NOT (b)-[:has_chunk]->(:Chunk)-[:cites]->(a)
        AND NOT (a)-[:cited_in]->(:Chunk)-[:cites]->(b)
        AND ID(a) > ID(b)
        RETURN a, r, b",
        "DELETE r",
        {batchSize:1000, parallel:false}
        )
        YIELD batches, total, timeTaken, committedOperations
        RETURN batches, total, timeTaken, committedOperations;""",
    )

In [37]:
pair_lst[0]['committedOperations']

556367